In [41]:
import time
import itertools
import re
import os
import pickle
import numpy as np

from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from lime.lime_text import LimeTextExplainer

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from collections import defaultdict




import tensorflow as tf

# PART 0: DATA LOADING AND PREPROCESSING

In [42]:

def clean_text(text):
    cleaned = re.sub(r'<.*?>', '', text).lower()
    return cleaned

def load_imdb_subset(
    num_samples=5000, 
    min_df=1, 
    max_features=15, 
    stopwords_option=True,
    stop_words = 'english'
):
    
    data = load_files(
        r"C:/Users/migue/Downloads/aclImdb_v1/aclImdb/train",
        categories=['pos','neg'], 
        encoding="utf-8", 
        decode_error="replace"                  
    )
    X_text_all, y_all = data.data, data.target


    X_text_all = [clean_text(txt) for txt in X_text_all]

    # Shuffle & truncate to num_samples
    full_idx = np.arange(len(X_text_all))
    #np.random.shuffle(full_idx)
    subset_idx = full_idx[:num_samples]
    global X_text 
    X_text = [X_text_all[i] for i in subset_idx]
    y = y_all[subset_idx]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_text, y, test_size=0.2, random_state=0
    )

    # Vectorizer: presence/absence
    if stopwords_option:
        vectorizer = CountVectorizer(
            binary=True, stop_words=stop_words, 
            min_df=min_df, max_features=max_features
        )
    else:
        vectorizer = CountVectorizer(
            binary=True, stop_words='english', 
            min_df=min_df, max_features=max_features
        )

    vectorizer.fit(X_train)
    return X_train, X_test, y_train, y_test, vectorizer



def train_NN_classifier(X_train, y_train, X_test, y_test, vectorizer):
    """
    Trains a neural network on the binary presence/absence of words.
    Returns the fitted model.
    """
    X_train_bow = vectorizer.transform(X_train)
    X_valid_bow = vectorizer.transform(X_test)
    input_dim = X_train_bow.shape[1]

    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),  # First hidden layer
        Dropout(0.3),  # Dropout with 30% probability
        Dense(32, activation='relu'),  # Second hidden layer
        Dropout(0.2),  # Dropout with 20% probability
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

    model.fit(X_train_bow, y_train, epochs=50, batch_size=20, validation_data=(X_valid_bow, y_test), verbose=1, callbacks=[early_stopping])
    return model

def get_cached_NN(X_train, y_train, vectorizer, num_samples, max_features, stop_words, X_valid, y_valid):

    filename = f"cached_classifier_ns{num_samples}_mf{max_features}_sw{stop_words}_NN_classifier_seed0.pkl"
    if os.path.exists(filename):
        print("Loading cached logistic from", filename)
        with open(filename, 'rb') as f:
            clNN = pickle.load(f)
    else:
        print("No cached classifier found. Training a new one...")
        clNN = train_NN_classifier(X_train, y_train, X_valid, y_valid, vectorizer)
        with open(filename, 'wb') as f:
            pickle.dump(clNN, f)
        print("Cached classifier saved as", filename)
    return clNN

def train_NN_classifier_filt(X_train_filt, y_train_filt, X_test_filt, y_test_filt, vectorizer_filt):
    """
    Trains a neural network on the binary presence/absence of words.
    Returns the fitted model.
    """
    X_train_bow_filt = vectorizer_filt.transform(X_train_filt)
    X_valid_bow_filt = vectorizer_filt.transform(X_test_filt)
    input_dim = X_train_bow_filt.shape[1]


    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),  # First hidden layer
        Dropout(0.3),  # Dropout with 30% probability
        Dense(32, activation='relu'),  # Second hidden layer
        Dropout(0.2),  # Dropout with 20% probability
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

    model.fit(X_train_bow_filt, y_train_filt, epochs=50, batch_size=20, validation_data=(X_valid_bow_filt, y_test_filt), verbose=1, callbacks=[early_stopping])
    return model

def get_cached_NN_filt(X_train_filt, y_train_filt, vectorizer_filt, num_samples, max_features, X_valid_filt, y_valid_filt):
    filename = f"cached_classifier_ns{num_samples}_mf{max_features}_sw_filtered_NN_classifier_seed0.pkl"
    if os.path.exists(filename):
        print("Loading cached logistic from", filename)
        with open(filename, 'rb') as f:
            clNN_filt = pickle.load(f)
    else:
        print("No cached classifier found. Training a new one...")
        clNN_filt = train_NN_classifier_filt(X_train_filt, y_train_filt, X_valid_filt, y_valid_filt, vectorizer_filt)
        with open(filename, 'wb') as f:
            pickle.dump(clNN_filt, f)
        print("Cached classifier saved as", filename)
    return clNN_filt



# CLASSICAL LIME

In [43]:
def run_classical_lime(
    text_sample, clNN, vectorizer,  
    k_features=20, num_samples=50   
):
    
    """
    Runs classical LIME on a single text instance.
    Returns the top (word, weight) pairs.
    """
    class_names = ["negative", "positive"]
    explainer = LimeTextExplainer(class_names=class_names, feature_selection="auto")

    def predict_proba(texts):
        bow = vectorizer.transform(texts) 
        #print('shaspe of box', bow.shape, ', text_sample:',text_sample, 'features: ', k_features, 'samples: ', num_samples)
        # print(bow)
        proba = clNN.predict(bow.toarray())
        if proba.ndim == 1:  # If 1D, reshape to (num_samples, 1)
            proba = proba.reshape(-1, 1)
        #print('proba', proba, 'dimension', proba.shape, 'return', np.hstack((1 - proba, proba)))
        return np.column_stack((1 - proba, proba))  # Return probabilities for both classes
        
        

    explanation = explainer.explain_instance(
        text_sample,
        predict_proba,
        num_features=k_features,
        num_samples=num_samples 
    )
    return explanation.as_list() 

# EXPERIMENTAL ROUTINE

In [44]:
def run_experiment( 
    num_samples=500,
    min_df=1,
    max_features=20,
    
    stopwords_option=True,
    lime_num_samples=300,
    stop_words = 'english',
    max_features_filt = 20

):
    
    X_train, X_test, y_train, y_test, vectorizer = load_imdb_subset(
        num_samples=num_samples,
        min_df=min_df,
        max_features=max_features,
        stopwords_option=stopwords_option,
        stop_words = stop_words
        )

    clNN = get_cached_NN(X_train, y_train, vectorizer, num_samples, max_features, stop_words, X_test, y_test)

    X_test_bow = vectorizer.transform(X_test)
    y_test = y_test.reshape(-1, 1)
    test_acc = accuracy_score(y_test, clNN.predict(X_test_bow) > 0.5)

    instance_local_accuracies = []

    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)
    X_all = X_train + X_test 
    y_all = np.concatenate([y_train, y_test.ravel()])



    word_weights_unf = defaultdict(list)

    for idx in range(len(X_all)):
        text_sample = X_all[idx]
        y_true = y_all[idx]


        explanation_lime_unfiltered = run_classical_lime(
            text_sample, clNN, vectorizer, 
            k_features=max_features, num_samples=lime_num_samples
        )

        bow = vectorizer.transform([text_sample])
        bin_features = bow.toarray()[0]

        contributions_unfiltered_abs = [(word, abs(score)) for word, score in explanation_lime_unfiltered]
        
        for word, weight in contributions_unfiltered_abs:
        
            word_weights_unf[word].append(weight)

        global_avg_weights_unf = {word: sum(weights) / len(weights) for word, weights in word_weights_unf.items()}

        threshold = 0.01
        

        filtered_words = {word: avg for word, avg in global_avg_weights_unf.items() if avg >= threshold}
        rubish_words = {word: avg for word, avg in global_avg_weights_unf.items() if avg <= threshold}
        #print("filtered_words", filtered_words)    #There is an issue here, the filtered words are not global, they are local. rubish should also be global

    print('filtered_words', filtered_words, 'rubish_words:', rubish_words)
    
    features_filt = len(global_avg_weights_unf) - len(rubish_words)
    
    X_train_filt, X_test_filt, y_train_filt, y_test_filt, vectorizer_filt = load_imdb_subset(
        num_samples=num_samples,
        min_df=min_df,
        max_features= max_features,
        stopwords_option=stopwords_option,
        stop_words = stop_words + list(rubish_words.keys())
        )
    
    # X_train_bow_filt = vectorizer_filt.transform(X_train_filt)
    # input_dim = X_train_bow_filt.shape[1]



    clNN_filtered = get_cached_NN_filt(X_train_filt, y_train, vectorizer_filt, num_samples, features_filt, X_test_filt, y_test)
    



    # print("Shape of training data:", X_train_filt.shape)
    # print("Shape of X_test_bow_filt:", X_test_bow_filt.shape)
    # print("Shape expected by model:", clNN_filtered.input_shape)
    # print("Shape of y_test_filt:", y_test_filt.shape)
    # print("Shape of predictions:", clNN_filtered.predict(X_test_bow_filt.toarray()).shape)
    # print("Model summary:")
    # clNN_filtered.summary()




    X_all_filt = X_train_filt + X_test_filt
    for idx in range(len(X_all_filt)):

        text_sample_filt = X_all_filt[idx]

        explanation_lime_filtered = run_classical_lime(
            text_sample_filt, clNN_filtered, vectorizer_filt, 
            k_features=features_filt, num_samples=lime_num_samples
        )
        
        bow_filt = vectorizer_filt.transform([text_sample_filt])

        y_pred = clNN_filtered.predict(bow_filt.toarray())[0].item()
        y_pred_label = 1 if y_pred >= 0.5 else 0

        instance_accuracy = int(y_pred_label == y_true)
        #print("instance_accuracy:", instance_accuracy)
        instance_local_accuracies.append(instance_accuracy)

        contributions_filtered_abs = [(word, abs(score)) for word, score in explanation_lime_filtered]
        
        #print("idx", idx, "text sample", text_sample)
     
        word_weights = defaultdict(list)

        for word, weight in contributions_filtered_abs:
            word_weights[word].append(weight)

        
    X_test_bow_filt = vectorizer.transform(X_test_filt)
    test_acc_filt = accuracy_score(y_test_filt, clNN_filtered.predict(X_test_bow_filt.toarray()) > 0.5)

    


    results = {
        "num_unf_words": len(word_weights_unf),
        "num_filt_words": len(word_weights),
        "local_accuracy": np.mean(instance_local_accuracies),
        "global_acc": np.mean(test_acc),
        "global_acc_filtered": np.mean(test_acc_filt),

        #"words_to_filter": rubish_words
    }
    return results



In [45]:
import pandas as pd
import sys, os

sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.getcwd()))

if __name__ == "__main__":


    # Parameter grid to systematically vary certain settings
    param_grid = {
        "num_samples": [200],
        "max_features": [15],
        "stopwords_option": [True],
        "lime_num_samples": [30],
        "stop_words": [['english']],
           
    }

    combos = list(itertools.product(*param_grid.values()))
    all_results = []

    for combo in combos:
        (num_samples_, max_features_, stopwords_, lime_samps_, stop_words_) = combo
        
        print("\n==================================")
        print(f"Running experiment with: "
              f"num_samples={num_samples_}, "
              f"max_features={max_features_}, "
              f"stopwords={stopwords_}, "
              f"lime_num_samples={lime_samps_}, "
              f"stop_words={stop_words_},")
        
        res = run_experiment(
            num_samples=num_samples_,
            max_features=max_features_,
            stopwords_option=stopwords_,
            lime_num_samples=lime_samps_,
            stop_words=stop_words_,
        )
        res_row = {
            "num_samples": num_samples_,
            "max_features": max_features_,
            "stopwords": stopwords_,
            "lime_num_samples": lime_samps_,
            "local_accuracy": res["local_accuracy"],
            "global_acc": res["global_acc"],
            "global_acc_filtered": res["global_acc_filtered"],
            "num_unf_words": res["num_unf_words"],
            "num_filt_words": res["num_filt_words"],
        }
        print("Results =>", res_row)
        all_results.append(res_row)

    # Save results to CSV
    df = pd.DataFrame(all_results)
    df.to_csv("results_expanded_flips.csv", index=False)
    print("\nAll done! Saved results to 'results_expanded_flips.csv'.")


Running experiment with: num_samples=200, max_features=15, stopwords=True, lime_num_samples=30, stop_words=['english'],
No cached classifier found. Training a new one...


c:\Users\migue\miniconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.5227 - loss: 0.6988 - val_accuracy: 0.7000 - val_loss: 0.6747
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4312 - loss: 0.7106 - val_accuracy: 0.7000 - val_loss: 0.6753
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5089 - loss: 0.7006 - val_accuracy: 0.7000 - val_loss: 0.6754
Cached classifier saved as cached_classifier_ns200_mf15_sw['english']_NN_classifier_seed0.pkl
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Shape of y_train: (160,)
Shape of y_test: (40, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━

c:\Users\migue\miniconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.4112 - loss: 0.7683 - val_accuracy: 0.6500 - val_loss: 0.6812
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4937 - loss: 0.7458 - val_accuracy: 0.6500 - val_loss: 0.6837
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4695 - loss: 0.7390 - val_accuracy: 0.6500 - val_loss: 0.6866
Cached classifier saved as cached_classifier_ns200_mf474_sw_filtered_NN_classifier_seed0.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step